In [6]:
from dotenv import load_dotenv
import pymongo
import os

load_dotenv()
CONNECTION_STRING = os.environ.get("COSMOS_CONNECTION_STRING")

In [7]:
client = pymongo.MongoClient(CONNECTION_STRING)

MemoryError: Cannot allocate write+execute memory for ffi.callback(). You might be running on a system that prevents this. For more information, see https://cffi.readthedocs.io/en/latest/using.html#callbacks

In [21]:
db = client['stock_data']
source_collection = db['all_stocks_ticks']
target_collection = db['600036.SH']

In [22]:
query = {'stock_code': '600036.SH'}
data = source_collection.find(query)
data = list(source_collection.find(query))

In [23]:
target_collection.insert_many(data)

InsertManyResult([ObjectId('6626375d54eac9463a3142f9'), ObjectId('6626375d54eac9463a3142fa'), ObjectId('6626375d54eac9463a3142fb'), ObjectId('6626375d54eac9463a3142fc'), ObjectId('6626375d54eac9463a3142fd'), ObjectId('6626375d54eac9463a3142fe'), ObjectId('6626375d54eac9463a3142ff'), ObjectId('6626375d54eac9463a314300'), ObjectId('6626375d54eac9463a314301'), ObjectId('6626375d54eac9463a314302'), ObjectId('6626375d54eac9463a314303'), ObjectId('6626375d54eac9463a314304'), ObjectId('6626375d54eac9463a314305'), ObjectId('6626375d54eac9463a314306'), ObjectId('6626375d54eac9463a314307'), ObjectId('6626375d54eac9463a314308'), ObjectId('6626375d54eac9463a314309'), ObjectId('6626375d54eac9463a31430a'), ObjectId('6626375d54eac9463a31430b'), ObjectId('6626375d54eac9463a31430c'), ObjectId('6626375d54eac9463a31430d'), ObjectId('6626375d54eac9463a31430e'), ObjectId('6626375d54eac9463a31430f'), ObjectId('6626375d54eac9463a314310'), ObjectId('6626375d54eac9463a314311'), ObjectId('6626375d54eac9463a3143

In [24]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext

In [25]:
sc = SparkContext("local[2]", "MongoDBStreaming")

24/04/23 19:48:32 WARN Utils: Your hostname, appledeMacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.79.41.17 instead (on interface en0)
24/04/23 19:48:32 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/04/23 19:48:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/04/23 19:48:33 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/04/23 19:48:33 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [26]:
ssc = StreamingContext(sc, 10)

/opt/homebrew/lib/python3.11/site-packages/pyspark/streaming/context.py:72: FutureWarning: DStream is deprecated as of Spark 3.4.0. Migrate to Structured Streaming.
  warnings.warn(


In [29]:
def get_mongodb_data():
    data = list(source_collection.find(query))
    return sc.parallelize(data)

In [31]:
mongo_stream = ssc.queueStream([get_mongodb_data()])

In [33]:
ssc.start()
ssc.awaitTermination()

24/04/23 19:53:20 ERROR JobScheduler: Error running job streaming job 1713873200000 ms.0
org.apache.spark.SparkException: An exception was raised by Python:
Traceback (most recent call last):
  File "/opt/homebrew/lib/python3.11/site-packages/pyspark/streaming/util.py", line 71, in call
    r = self.func(t, *rdds)
        ^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/lib/python3.11/site-packages/pyspark/streaming/dstream.py", line 254, in takeAndPrint
    taken = rdd.take(num + 1)
            ^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/lib/python3.11/site-packages/pyspark/rdd.py", line 2855, in take
    res = self.context.runJob(self, takeUpToNumLeft, p)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/lib/python3.11/site-packages/pyspark/context.py", line 2510, in runJob
    sock_info = self._jvm.PythonRDD.runJob(self._jsc.sc(), mappedRDD._jrdd, partitions)
                                                           ^^^^^^^^^^^^^^^
  File "/opt/homebrew/lib/python3

Py4JJavaError: An error occurred while calling o25.awaitTermination.
: org.apache.spark.SparkException: An exception was raised by Python:
Traceback (most recent call last):
  File "/opt/homebrew/lib/python3.11/site-packages/pyspark/streaming/util.py", line 71, in call
    r = self.func(t, *rdds)
        ^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/lib/python3.11/site-packages/pyspark/streaming/dstream.py", line 254, in takeAndPrint
    taken = rdd.take(num + 1)
            ^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/lib/python3.11/site-packages/pyspark/rdd.py", line 2855, in take
    res = self.context.runJob(self, takeUpToNumLeft, p)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/lib/python3.11/site-packages/pyspark/context.py", line 2510, in runJob
    sock_info = self._jvm.PythonRDD.runJob(self._jsc.sc(), mappedRDD._jrdd, partitions)
                                                           ^^^^^^^^^^^^^^^
  File "/opt/homebrew/lib/python3.11/site-packages/

24/04/23 19:53:30 ERROR JobScheduler: Error running job streaming job 1713873210000 ms.0
org.apache.spark.SparkException: An exception was raised by Python:
Traceback (most recent call last):
  File "/opt/homebrew/lib/python3.11/site-packages/pyspark/streaming/util.py", line 71, in call
    r = self.func(t, *rdds)
        ^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/lib/python3.11/site-packages/pyspark/streaming/dstream.py", line 254, in takeAndPrint
    taken = rdd.take(num + 1)
            ^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/lib/python3.11/site-packages/pyspark/rdd.py", line 2855, in take
    res = self.context.runJob(self, takeUpToNumLeft, p)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/lib/python3.11/site-packages/pyspark/context.py", line 2510, in runJob
    sock_info = self._jvm.PythonRDD.runJob(self._jsc.sc(), mappedRDD._jrdd, partitions)
                                                           ^^^^^^^^^^^^^^^
  File "/opt/homebrew/lib/python3